In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.nn import MSELoss
from mediapipe import solutions

from utils import smooth_data, split_data
from video_converter import Video2DataFrame
from custom_pose_landmarks import CustomPoseLandmark
from model_builder import RNN
from engine import train
from data_setup import create_dataloaders

from torch.utils.data import BatchSampler, DataLoader, Dataset
from custom_dataset_2 import CustomDataset, PaddedBatchSampler, floor_ceil, add_padding, collate_fn

In [2]:
# Prepare paths
data_path = '../data/'
video_path = os.path.join(data_path, 'raw/squat')

# Read personal data from excel
personal_data = pd.read_excel(os.path.join(data_path, 'PersonalData.xlsx'))

In [3]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 31, 32]

# Custom pose landmark names and their connections
landmarks = {
    'THORAX': ['NOSE'],
    'PELVIS': ['LEFT_HIP', 'RIGHT_HIP'],}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom_pose = CustomPoseLandmark(mp_pose, values, landmarks)

In [ ]:
converter = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

# Convert videos to dataframe
dataframe = converter.get_dataframe(
    source=video_path,
    detection=0.9,
    tracking=0.9,
    video_display=False
)

In [ ]:
# dataframe.to_csv('ConvertedSquats.csv', index=False)

In [4]:
dataframe = pd.read_csv('ConvertedSquats.csv')

In [5]:
# Merge personal and video data
data = pd.merge(dataframe, personal_data, on='Id')

# Calculate the maximum load that was passed
max_load = data.loc[data['Lifted'] == 1, ['Id', 'Load']].groupby(by='Id', as_index=False).max()
max_load = max_load.rename(columns={'Load': 'MaxLoad'})
data = pd.merge(data, max_load, on='Id')

# Calculate what percentage of the maximum load is the current load
data['PercentageMaxLoad'] = 100 * data['Load'] / data['MaxLoad']

del data['MaxLoad']

# Get only lifted approaches
data = data.loc[data['Lifted'] == 1]

# Variables that aren't needed in the first run
to_drop = [
    'Id', 'Age', 'Height', 'Weight', 'PastInjuries', 'LastInjury', 'PainDuringTraining', 'SquatRecord',
    'BenchPressRecord', 'DeadliftRecord', 'PhysicalActivities', 'SetNumber', 'Load', 'Lifted', 'Timestamp']

data = data.drop(columns=to_drop)

# Categorical variables that need to be one hot encoded
to_one_hot = [
    'ProficiencyLevel', 'EquipmentAvailability', 'TrainingProgram', 'TrainingFrequency', 'CameraPosition']

data = pd.get_dummies(data, columns=to_one_hot, dtype=int)

# Move the PercentageMaxLoad column to the end of the dataframe
percentage = data.pop('PercentageMaxLoad')
data['PercentageMaxLoad'] = percentage

# Smooth all features extracted from MediaPipe solution
data = smooth_data(data, frac=0.1, it=3)

In [18]:
# Setup hyperparameters
BATCH_SIZE = 128
NUM_WORKERS = 0
PIN_MEMORY = True

custom_dataset = CustomDataset(data)

batch_sampler = PaddedBatchSampler(custom_dataset, batch_size=BATCH_SIZE)

data_loader = DataLoader(
    custom_dataset, batch_sampler=batch_sampler, collate_fn=collate_fn)

In [22]:
next(iter(data_loader))

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
# Get dictionary of file ids assingned to different datasets
file_ids = split_data(data)

# Setup hyperparameters
BATCH_SIZE = 128
NUM_WORKERS = 0
PIN_MEMORY = True

train_dataloader, valid_dataloader, test_dataloader = create_dataloaders(
    data=data,
    file_ids=file_ids,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY
)

In [ ]:
# Setup hyperparameters
INPUT_SIZE = 78
HIDDEN_SIZE = 512
NUM_LAYERS = 5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_EPOCHS = 50
LEARNING_RATE = 0.001

# Initialize RNN model class instance
model = RNN(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS)

# Send model to device
model = model.to(device=DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = MSELoss()

In [ ]:
results = train(model, train_dataloader, valid_dataloader, optimizer, loss_fn, NUM_EPOCHS, DEVICE)

In [ ]:
plt.plot(range(1, NUM_EPOCHS + 1), results['train_loss']);
plt.plot(range(1, NUM_EPOCHS + 1), results['valid_loss']);
plt.legend(['train loss curve', 'valid loss curve'])

In [ ]:
# Put model in evaluation mode
model.eval()

# Turn on inference context manager
with torch.inference_mode():
    # Loop through DataLoader batches
    for data, targets in train_dataloader:
        # Send data to target device
        data, targets = data.to(DEVICE), targets.to(DEVICE)

        # Forward pass
        predictions = model(data)

        # Calculate and accumulate loss
        loss = loss_fn(targets, predictions)

In [ ]:
pd.DataFrame({'targets': targets.cpu().squeeze(), 'predictions': predictions.cpu().squeeze()}, index=range(len(targets.cpu())))